# HOMEWORK 1 - Chess Game
**Student:** Alessandro Mattei

**Matricola:** 295411

**Email:** alessandro.mattei1@student.univaq.it

The main components used for the implementation will be presented:
   - Class Agent
   - Class Game (ChessGame)
   - Class State (StateChessGame)
   - Heuristics (HardBoardEvaluationChessGame and SoftBoardEvaluationChessGame)
   - Observation (ObservationBoard) 
   - Search Algorithm (MinMaxAplaBetaPruning, MinMaxAlphaBetaPruningH0Cut, MinMaxAlphaBetaPruningHlCut, MinMaxAlphaBetaPruningRegressorCut)
   - Nonlinear Regressor
   - CSV generator for the nonlinear regressor

# Agent Class
Represents an agent that can act based on a given search algorithm and its current view of the world.

In [3]:
class Agent:
    """
    Represents an agent that can act based on a given search algorithm and its current view of the world.

    Attributes:
        search_algorithm: A search algorithm that the agent uses to make decisions.
        view: The agent's current view of the world.
        old_view: The agent's previous view of the world.
    """

    def __init__(self, search_algorithm, initial_state):
        """
        Initializes the Agent with a search algorithm and an initial state.

        :param search_algorithm: The search algorithm to be used by the agent.
        :param initial_state: The initial state of the world as perceived by the agent.
        """
        self.search_algorithm = search_algorithm
        self.view = initial_state
        self.old_view = None

    def do_action(self, current_state_world):
        """
        Updates the agent's view based on the current state of the world and the search algorithm.
        :param current_state_world: The current state of the world.
        :return: The updated view of the agent.
        """
        self.view = self.search_algorithm.search(current_state_world)
        self.old_view = current_state_world
        return self.view


# State
Represents a state in a chess game, including the board configuration and various heuristic evaluations.

In [5]:
import chess


class StateChessGame:
    """
    Represents a state in a chess game, including the board configuration and various heuristic evaluations.

    Attributes:
        game_board (chess.Board): The current chess board configuration.
        parent_state (StateChessGame): The parent state from which this state is derived.
        move (chess.Move): The move that led to this state.
        h (float): General heuristic value for the state.
        h0 (float): Heuristic value used for h0 cutoff.
        hl (float): Heuristic value used for hl cutoff.
        hr (float): Heuristic value used for nonlinear regressor cutoff.
    """

    def __init__(self, game_board=None, state_parent=None, move=None):
        """
        Initializes a new game state.

        :param game_board: The current chess board configuration. If None, initializes a new chess board.
        :param state_parent: The parent state from which this state is derived.
        :param move: The move that led to this state.
        """
        self.game_board = game_board  # The current chess board (chess.Board object).
        self.parent_state = state_parent  # The parent state from which this state is derived.
        self.move = move  # The move that led to this state.
        self.h = None  # General heuristic value for the state.
        self.h0 = None  # Heuristic value used for h0 cutoff.
        self.hl = None  # Heuristic value used for hl cutoff.
        self.hr = None  # Heuristic value used for nonlinear regressor cutoff.

        # If no game board is provided, initialize a new chess board.
        if self.game_board is None:
            self.game_board = chess.Board()

    def __eq__(self, other):
        """
        Checks if this state is equal to another state. States are considered equal if they have the same game
        board configuration.

        :param other: The other StateChessGame object to compare with.
        :return: True if the states are equal, False otherwise.
        """
        if not isinstance(other, StateChessGame):
            return False
        return self.game_board == other.game_board

    def __ne__(self, other):
        """
        Checks if this state is not equal to another state. It relies on the __eq__ method.

        :param other: The other StateChessGame object to compare with.
        :return: True if the states are not equal, False otherwise.
        """
        return not self.__eq__(other)

    def __hash__(self):
        """
        Generates a hash for the state. This is based on the string representation of the game board, allowing the state
        to be used in hash tables or sets.

        :return: The hash of the state.
        """
        return hash(str(self.game_board))


# Game Class
Represents a chess game, providing functionalities to manage the game state and compute possible moves.

In [6]:
import chess

from chessgame.StateChessGame import StateChessGame


class ChessGame:
    """
    Represents a chess game, providing functionalities to manage the game state and compute possible moves.

    Attributes:
        game_board (chess.Board): The current chess board configuration.
    """

    def __init__(self, game_board=None):
        """
        Initializes a new chess game.

        :param game_board: The current chess board configuration. If None, initializes a new chess board.
        """
        self.game_board = game_board  # The current chess board.

        # If no game board is provided, initialize a new chess board.
        if game_board is None:
            self.game_board = chess.Board()

    def neighbors(self, state: StateChessGame):
        """
        Generates all possible next states (neighbors) from a given state.

        :param state: The current state of the chess game from which to compute neighbors.
        :return: A list of StateChessGame objects representing possible next states.
        """
        neighbors = []

        # Iterate through all legal moves from the current state.
        for legal_move in state.game_board.legal_moves:
            # Copy the current game board and make the legal move.
            new_game_board = state.game_board.copy()
            new_game_board.push(legal_move)

            # Create a new StateChessGame object for the resulting game state.
            neighbor = StateChessGame(game_board=new_game_board, state_parent=state, move=legal_move)
            neighbors.append(neighbor)
        return neighbors

    def get_name_winner_player(self, game_board):
        """
        Determines the name of the winning player if the game is in checkmate.

        :param game_board: The chess board to check for checkmate and winner.
        :return: The name of the winning player ("White" or "Black") if there's a checkmate, otherwise None.
        """
        # Check if the current game state is a checkmate.
        if game_board.is_checkmate():
            # Get the outcome of the game.
            outcome = game_board.outcome()
            if outcome is not None:
                # Return "White" or "Black" depending on the winner.
                return "White" if outcome.winner else "Black"
        return None


# Heuristics
## Single Evaluations
### Constants

In [ ]:
import chess

PIECE_VALUE = {
    chess.PAWN: 1,
    chess.KNIGHT: 3,
    chess.BISHOP: 3,
    chess.ROOK: 5,
    chess.QUEEN: 9,
    chess.KING: 0  # Il valore del re è gestito separatamente
}

# Tabelle di posizione per il pedone
PAWN_TABLE = [
    0, 0, 0, 0, 0, 0, 0, 0,
    5, 10, 10, -20, -20, 10, 10, 5,
    5, -5, -10, 0, 0, -10, -5, 5,
    0, 0, 0, 20, 20, 0, 0, 0,
    5, 5, 10, 25, 25, 10, 5, 5,
    10, 10, 20, 30, 30, 20, 10, 10,
    50, 50, 50, 50, 50, 50, 50, 50,
    0, 0, 0, 0, 0, 0, 0, 0
]

# Tabelle di posizione per il cavallo
KNIGHT_TABLE = [
    -50, -40, -30, -30, -30, -30, -40, -50,
    -40, -20, 0, 5, 5, 0, -20, -40,
    -30, 5, 10, 15, 15, 10, 5, -30,
    -30, 0, 15, 20, 20, 15, 0, -30,
    -30, 5, 15, 20, 20, 15, 5, -30,
    -30, 0, 10, 15, 15, 10, 0, -30,
    -40, -20, 0, 0, 0, 0, -20, -40,
    -50, -40, -30, -30, -30, -30, -40, -50,
]

# Tabelle di posizione per l'alfiere
BISHOP_TABLE = [
    -20, -10, -10, -10, -10, -10, -10, -20,
    -10, 5, 0, 0, 0, 0, 5, -10,
    -10, 10, 10, 10, 10, 10, 10, -10,
    -10, 0, 10, 10, 10, 10, 0, -10,
    -10, 5, 5, 10, 10, 5, 5, -10,
    -10, 0, 5, 10, 10, 5, 0, -10,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -20, -10, -10, -10, -10, -10, -10, -20
]

# Tabelle di posizione per la torre
ROOK_TABLE = [
    0, 0, 0, 5, 5, 0, 0, 0,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    5, 10, 10, 10, 10, 10, 10, 5,
    0, 0, 0, 0, 0, 0, 0, 0
]

# Tabelle di posizione per la regina
QUEEEN_TABLE = [
    -20, -10, -10, -5, -5, -10, -10, -20,
    -10, 0, 5, 0, 0, 0, 0, -10,
    -10, 5, 5, 5, 5, 5, 0, -10,
    0, 0, 5, 5, 5, 5, 0, -5,
    -5, 0, 5, 5, 5, 5, 0, -5,
    -10, 0, 5, 5, 5, 5, 0, -10,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -20, -10, -10, -5, -5, -10, -10, -20
]

# Tabelle di posizione per il re (inizio gioco)
KING_INITGAME_TABLE = [
    20, 30, 10, 0, 0, 10, 30, 20,
    20, 20, 0, 0, 0, 0, 20, 20,
    -10, -20, -20, -20, -20, -20, -20, -10,
    -20, -30, -30, -40, -40, -30, -30, -20,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30
]

# Tabelle di posizione per il re (fine gioco)
KING_ENDGAME_TABLE = [
    -50, -40, -30, -20, -20, -30, -40, -50,
    -30, -20, -10, 0, 0, -10, -20, -30,
    -30, -10, 20, 30, 30, 20, -10, -30,
    -30, -10, 30, 40, 40, 30, -10, -30,
    -30, -10, 30, 40, 40, 30, -10, -30,
    -30, -10, 20, 30, 30, 20, -10, -30,
    -30, -30, 0, 0, 0, 0, -30, -30,
    -50, -30, -30, -30, -30, -30, -30, -50
]

### Evaluate Board Without King

Provides heuristic evaluation of a chess board state, focusing on piece values and game conditions.

In [ ]:
class EvaluateBoardWithoutKing:
    """
    Provides heuristic evaluation of a chess board state, focusing on piece values and game conditions.

    Attributes:
        evaluate_end_game_phase (bool): Flag to indicate whether to evaluate endgame phases differently.
        normalize_result (bool): Flag to indicate whether to normalize the evaluation result.
        h_max_value (int): Maximum heuristic value for normalization.
        h_min_value (int): Minimum heuristic value for normalization.
    """

    def __init__(self, evaluate_end_game_phase=False, normalize_result=False):
        """
        Initializes the evaluator with options for endgame evaluation and result normalization.

        :param evaluate_end_game_phase: Set to True to evaluate endgame phases differently.
        :param normalize_result: Set to True to normalize the evaluation result.
        """
        self.evaluate_end_game_phase = evaluate_end_game_phase  # Flag to evaluate endgame phases.
        self.normalize_result = normalize_result  # Flag to normalize the evaluation result.
        self.h_max_value = 99  # Maximum heuristic value for normalization.
        self.h_min_value = -99  # Minimum heuristic value for normalization.

    def h(self, state: StateChessGame):
        """
        Evaluates the heuristic of a given game state.

        :param state: StateChessGame object representing the current state of the chess game.
        :return: The heuristic value of the state.
        """
        # Evaluates endgame phase or normalizes the result based on the flags set in the constructor.
        if self.evaluate_end_game_phase:
            return self.__h(state.game_board)
        elif self.normalize_result:
            raw_eval = self.__h(state.game_board)
            return self.__normalize(raw_eval)
        else:
            return self.__h(state.game_board)

    def h_piccolo(self, board):
        """
        Similar to h() but operates directly on a chess board and allows specifying normalization bounds.

        :param board: The chess board to evaluate.
        :return: The heuristic value of the board.
        """
        # Evaluates the endgame phase, normalizes the result, or provides raw evaluation.
        if self.evaluate_end_game_phase:
            return self.__h(board)
        elif self.normalize_result:
            raw_eval = self.__h(board)
            return self.__normalize(raw_eval, 10, -10)
        else:
            return self.__h(board)

    def __h(self, board):
        """
        Private method for raw heuristic evaluation of a board.

        :param board: The chess board to evaluate.
        :return: The raw heuristic value of the board.
        """
        # Special handling for endgame phase.
        if self.evaluate_end_game_phase:
            game_over_eval = None
            # Assign extreme values for checkmate situations.
            if board.is_checkmate():
                outcome = board.outcome()
                if outcome is not None:
                    game_over_eval = float("inf") if outcome.winner else float("-inf")
            # Assign zero for draw situations.
            if board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
                game_over_eval = 0

            if game_over_eval is not None:
                return game_over_eval

        # If the game is over, returns high positive or negative values for checkmate, and zero for other endings.
        if board.is_game_over():
            if board.is_checkmate():
                return -99 if board.turn else 99
            else:
                return 0  # Handles stalemate and insufficient material.

        # Piece-based evaluation, optimized.
        eval = sum(PIECE_VALUE[piece] * (len(board.pieces(piece, chess.WHITE)) - len(board.pieces(piece, chess.BLACK)))
                   for piece in PIECE_VALUE)
        # Slightly favors the player whose turn it is, as they might have the initiative.
        eval += 0.1 if board.turn else -0.1

        return eval

    def __normalize(self, value, maxv=100, minv=-100):
        """
        Normalizes the evaluation value within a specified range.

        :param value: The value to be normalized.
        :param maxv: The maximum value for normalization. Defaults to 100.
        :param minv: The minimum value for normalization. Defaults to -100.
        :return: The normalized value.
        """
        # Normalizes the value within the range from minv to maxv.
        if value >= 0:
            # Normalizes positive values.
            normalized = (value / self.h_max_value) * 100
        else:
            # Normalizes negative values.
            normalized = (value / abs(self.h_min_value)) * 100

        # Limits the normalized value between minv and maxv.
        normalized = max(min(normalized, maxv), minv)
        return normalized


## Evaluate Central Control Score

Provides heuristic evaluation of a chess board state with a focus on the control of central squares.

In [ ]:
class EvaluateCentralControlScore:
    """
    Provides heuristic evaluation of a chess board state with a focus on the control of central squares.

    Attributes:
        evaluate_end_game_phase (bool): Flag to indicate whether to evaluate endgame phases differently.
        normalize_result (bool): Flag to indicate whether to normalize the evaluation result.
        h_max_value (float): Maximum heuristic value for normalization.
        h_min_value (float): Minimum heuristic value for normalization.
    """

    def __init__(self, evaluate_end_game_phase=False, normalize_result=False):
        """
        Initializes the evaluator with options for endgame evaluation and result normalization.

        :param evaluate_end_game_phase: Set to True to evaluate endgame phases differently.
        :param normalize_result: Set to True to normalize the evaluation result.
        """
        self.evaluate_end_game_phase = evaluate_end_game_phase  # Flag to evaluate endgame phases.
        self.normalize_result = normalize_result  # Flag to normalize the evaluation result.
        self.h_max_value = 1.2  # Maximum heuristic value for normalization.
        self.h_min_value = -1.2  # Minimum heuristic value for normalization.

    def h(self, state: StateChessGame):
        """
        Evaluates the heuristic of a given game state.

        :param state: StateChessGame object representing the current state of the chess game.
        :return: The heuristic value of the state.
        """
        # Evaluates endgame phase or normalizes the result based on the constructor's flags.
        if self.evaluate_end_game_phase:
            return self.__h(state.game_board)
        elif self.normalize_result:
            raw_eval = self.__h(state.game_board)
            return self.__normalize(raw_eval)
        else:
            return self.__h(state.game_board)

    def h_piccolo(self, board):
        """
        Similar to h() but operates directly on a chess board and allows specifying normalization bounds.

        :param board: The chess board to evaluate.
        :return: The heuristic value of the board.
        """
        # Evaluates the endgame phase, normalizes the result, or provides raw evaluation.
        if self.evaluate_end_game_phase:
            return self.__h(board)
        elif self.normalize_result:
            raw_eval = self.__h(board)
            return self.__normalize(raw_eval, 10, -10)
        else:
            return self.__h(board)

    def __h(self, board):
        """
        Private method for raw heuristic evaluation of a board.

        :param board: The chess board to evaluate.
        :return: The raw heuristic value of the board.
        """
        # Special handling for endgame phase.
        if self.evaluate_end_game_phase:
            game_over_eval = None
            # Assign extreme values for checkmate situations.
            if board.is_checkmate():
                outcome = board.outcome()
                if outcome is not None:
                    game_over_eval = float("inf") if outcome.winner else float("-inf")
            # Assign zero for draw situations.
            if board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
                game_over_eval = 0

            if game_over_eval is not None:
                return game_over_eval

        # Assign points for control of each central square.
        center_squares = [chess.D4, chess.E4, chess.D5, chess.E5]
        score = 0
        central_square_control = 0.3  # Value to calibrate based on your specific heuristic.

        # Check if central squares are controlled by White or Black.
        for square in center_squares:
            if board.is_attacked_by(chess.WHITE, square):
                score += central_square_control
            if board.is_attacked_by(chess.BLACK, square):
                score -= central_square_control

        # Adjust the score for the current player.
        return score if board.turn == chess.WHITE else -score

    def __normalize(self, value, maxv=100, minv=-100):
        """
        Normalizes the evaluation value within a specified range.

        :param value: The value to be normalized.
        :param maxv: The maximum value for normalization. Defaults to 100.
        :param minv: The minimum value for normalization. Defaults to -100.
        :return: The normalized value.
        """
        # Normalizes the value within the range from minv to maxv.
        if value >= 0:
            # Normalizes positive values.
            normalized = (value / self.h_max_value) * 100
        else:
            # Normalizes negative values.
            normalized = (value / abs(self.h_min_value)) * 100

        # Limits the normalized value between minv and maxv.
        normalized = max(min(normalized, maxv), minv)
        return normalized

## Evaluate King Safety

A class to evaluate the safety of the king in a chess game. It assesses the level of threat or safety for both kings based on the game board configuration.

In [ ]:
class EvaluateKingSafety:
    """
    A class to evaluate the safety of the king in a chess game. It assesses the level of threat or safety
    for both kings based on the game board configuration.

    Attributes:
        evaluate_end_game_phase (bool): Flag to indicate if endgame phases should be evaluated differently.
        normalize_result (bool): Flag to determine if the evaluation result should be normalized.
        h_max_value (float): Maximum heuristic value for normalization purposes.
        h_min_value (float): Minimum heuristic value for normalization purposes.
    """

    def __init__(self, evaluate_end_game_phase=False, normalize_result=False):
        """
        Initializes the evaluator with options for endgame evaluation and result normalization.

        :param evaluate_end_game_phase: Set to True to apply special evaluations in endgame phases.
        :param normalize_result: Set to True to normalize the evaluation score within a specific range.
        """
        self.evaluate_end_game_phase = evaluate_end_game_phase
        self.normalize_result = normalize_result
        self.h_max_value = 9.5  # Max heuristic value for normalization.
        self.h_min_value = -9.5  # Min heuristic value for normalization

    def h(self, state: StateChessGame):
        """
        Evaluates the safety of the king based on the current game state.

        :param state: StateChessGame object representing the current state of the chess game.
        :return: The heuristic value representing the king's safety.
        """
        # Applies special endgame evaluation or normalization as per the initialization flags.
        if self.evaluate_end_game_phase:
            return self.__h(state.game_board)
        elif self.normalize_result:
            raw_eval = self.__h(state.game_board)
            return self.__normalize(raw_eval)
        else:
            return self.__h(state.game_board)

    def h_piccolo(self, board):
        """
        A variant of the h() method, working directly on a chess board.

        :param board: The chess board to evaluate.
        :return: The heuristic value representing the king's safety.
        """
        # Handles endgame phase evaluation or normalization as specified.
        if self.evaluate_end_game_phase:
            return self.__h(board)
        elif self.normalize_result:
            raw_eval = self.__h(board)
            return self.__normalize(raw_eval, 10, -10)
        else:
            return self.__h(board)

    def __h(self, board):
        """
        Private method for raw heuristic evaluation of the king's safety on the board.

        :param board: The chess board to evaluate.
        :return: The raw heuristic value representing the king's safety.
        """
        # Special handling for endgame phase.
        if self.evaluate_end_game_phase:
            game_over_eval = None
            # Assign extreme values for checkmate situations.
            if board.is_checkmate():
                outcome = board.outcome()
                if outcome is not None:
                    game_over_eval = float("inf") if outcome.winner else float("-inf")
            # Assign zero for draw situations.
            if board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
                game_over_eval = 0

            if game_over_eval is not None:
                return game_over_eval
        score = 0
        pawn_cover_score = 0.5
        attacked_square_score = -0.75

        # Bit masks for black and white pedestrians
        white_pawns = board.pieces(chess.PAWN, chess.WHITE)
        black_pawns = board.pieces(chess.PAWN, chess.BLACK)

        # Calculation for the white king
        white_king_square = board.king(chess.WHITE)
        white_king_attacks = board.attacks(white_king_square)
        white_king_zone = white_pawns & white_king_attacks
        score += pawn_cover_score * bin(white_king_zone).count('1')

        # Controlla le caselle attaccate dai neri nella zona del re bianco
        for square in white_king_attacks:
            if board.is_attacked_by(chess.BLACK, square):
                score += attacked_square_score

        # Check the squares attacked by blacks in the white king's area
        black_king_square = board.king(chess.BLACK)
        black_king_attacks = board.attacks(black_king_square)
        black_king_zone = black_pawns & black_king_attacks
        score -= pawn_cover_score * bin(black_king_zone).count('1')

        # Check the squares attacked by whites in the black king's area
        for square in black_king_attacks:
            if board.is_attacked_by(chess.WHITE, square):
                score -= attacked_square_score

        return score

    def __normalize(self, value, maxv=100, minv=-100):
        """
        Normalizes the evaluation value within a specified range.

        :param value: The value to be normalized.
        :param maxv: The maximum value for normalization. Defaults to 100.
        :param minv: The minimum value for normalization. Defaults to -100.
        :return: The normalized value.
        """
        # Normalizes the value within the range from minv to maxv.
        if value >= 0:
            # Normalizes positive values.
            normalized = (value / self.h_max_value) * 100
        else:
            # Normalizes negative values.
            normalized = (value / abs(self.h_min_value)) * 100

        # Limits the normalized value between minv and maxv.
        normalized = max(min(normalized, maxv), minv)
        return normalized


## Evaluate Mobility

This class evaluates the mobility of pieces in a chess game. It calculates a score based on the number of legal moves available for white and black, representing the degree of freedom and potential for offensive or defensive actions in the game.

In [ ]:
class EvaluateMobility:
    """
    This class evaluates the mobility of pieces in a chess game. It calculates a score based on
    the number of legal moves available for white and black, representing the degree of freedom
    and potential for offensive or defensive actions in the game.

    Attributes:
        evaluate_end_game_phase (bool): Indicates if the endgame should be evaluated differently.
        normalize_result (bool): Determines if the evaluation score should be normalized.
        h_max_value (float): The upper limit for normalization of the heuristic score.
        h_min_value (float): The lower limit for normalization of the heuristic score.
    """

    def __init__(self, evaluate_end_game_phase=False, normalize_result=False):
        """
        Initializes the evaluator with options for endgame evaluation and result normalization.

        :param evaluate_end_game_phase: Set to True for specialized evaluations in endgame phases.
        :param normalize_result: Set to True to normalize the evaluation score within a range.
        """
        self.evaluate_end_game_phase = evaluate_end_game_phase
        self.normalize_result = normalize_result
        self.h_max_value = 90  # Max heuristic value for normalization.
        self.h_min_value = -10  # Min heuristic value for normalization.

    def h(self, state: StateChessGame):
        """
        Evaluates the mobility based on the current game state. Determines if special handling
        for the endgame or normalization of results is required.

        :param state: StateChessGame object representing the current state of the chess game.
        :return: A heuristic value representing the mobility balance.
        """
        # Applies endgame evaluation or normalization based on the initialization flags.
        if self.evaluate_end_game_phase:
            return self.__h(state.game_board)
        elif self.normalize_result:
            raw_eval = self.__h(state.game_board)
            return self.__normalize(raw_eval)
        else:
            return self.__h(state.game_board)

    def h_piccolo(self, board):
        """
        A similar function to h(), but operates directly on a chess board and allows for custom
        normalization bounds.

        :param board: The chess board to evaluate.
        :return: A heuristic value representing the mobility balance.
        """
        # Handles endgame evaluation or normalization based on the board state.
        if self.evaluate_end_game_phase:
            return self.__h(board)
        elif self.normalize_result:
            raw_eval = self.__h(board)
            return self.__normalize(raw_eval, 10, -10)
        else:
            return self.__h(board)

    def __h(self, board):
        """
        Private method for the raw heuristic evaluation of mobility on the board. It calculates
        the difference in the number of legal moves available to each player.

        :param board: The chess board to evaluate.
        :return: The raw heuristic value representing the mobility balance.
        """
        # Special handling for endgame phase.
        if self.evaluate_end_game_phase:
            game_over_eval = None
            # Assign extreme values for checkmate situations.
            if board.is_checkmate():
                outcome = board.outcome()
                if outcome is not None:
                    game_over_eval = float("inf") if outcome.winner else float("-inf")
            # Assign zero for draw situations.
            if board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
                game_over_eval = 0

            if game_over_eval is not None:
                return game_over_eval
        white_mobility = 0
        black_mobility = 0

        # Calculate the mobility for both white and black pieces.
        for move in board.legal_moves:
            if board.color_at(move.from_square) == chess.WHITE:
                white_mobility += 1
            else:
                black_mobility += 1

        mobility_balance = white_mobility - black_mobility
        # Return the mobility balance, adjusted for the current player's turn.
        return mobility_balance if board.turn == chess.WHITE else -mobility_balance

    def __normalize(self, value, maxv=100, minv=-100):
        """
        Normalizes the evaluation value within a specified range.

        :param value: The value to be normalized.
        :param maxv: The maximum value for normalization. Defaults to 100.
        :param minv: The minimum value for normalization. Defaults to -100.
        :return: The normalized value.
        """
        # Normalizes the value within the range from minv to maxv.
        if value >= 0:
            # Normalizes positive values.
            normalized = (value / self.h_max_value) * 100
        else:
            # Normalizes negative values.
            normalized = (value / abs(self.h_min_value)) * 100

        # Limits the normalized value between minv and maxv.
        normalized = max(min(normalized, maxv), minv)
        return normalized


## Evaluate Pawn Structure

This class evaluates the pawn structure in a chess game. It analyzes various factors like isolated, doubled, backward, and passed pawns, computing a score to represent the strategic and positional strength of the pawn structure.

In [ ]:
class EvaluatePawnStructure:
    """
    This class evaluates the pawn structure in a chess game. It analyzes various factors like isolated,
    doubled, backward, and passed pawns, computing a score to represent the strategic and positional strength
    of the pawn structure.

    Attributes:
        evaluate_end_game_phase (bool): Indicates if the endgame should be evaluated differently.
        file_bb (list): Bitboards representing each file on the chessboard.
        rank_bb (list): Bitboards representing each rank on the chessboard.
        advance_shifts (dict): Dict for calculating the square index after advancing a pawn.
        normalize_result (bool): Determines if the evaluation score should be normalized.
        h_max_value (float): The upper limit for normalization of the heuristic score.
        h_min_value (float): The lower limit for normalization of the heuristic score.
    """

    def __init__(self, evaluate_end_game_phase=False, normalize_result=False):
        """
        Initializes the evaluator with options for endgame evaluation, pawn advancement, and result normalization.

        :param evaluate_end_game_phase: Set to True for specialized evaluations in endgame phases.
        :param normalize_result: Set to True to normalize the evaluation score within a range.
        """
        self.evaluate_end_game_phase = evaluate_end_game_phase  # Flag to adjust evaluation in endgame phases.
        self.file_bb = [chess.BB_FILES[f] for f in range(8)]  # Bitboards for each file (column) of the chessboard.
        self.rank_bb = [chess.BB_RANKS[r] for r in range(8)]  # Bitboards for each rank (row) of the chessboard.
        self.advance_shifts = {chess.WHITE: 8, chess.BLACK: -8}  # Square shift for pawn advancement based on color.
        self.normalize_result = normalize_result  # Flag to normalize the evaluation score.
        self.h_max_value = 430  # Max heuristic value for normalization.
        self.h_min_value = -430  # Min heuristic value for normalization.

    def h(self, state: StateChessGame):
        """
        Evaluates the pawn structure based on the current game state. Determines if special handling
        for the endgame or normalization of results is required.

        :param state: StateChessGame object representing the current state of the chess game.
        :return: A heuristic value representing the pawn structure.
        """
        # Applies endgame evaluation or normalization based on the initialization flags.
        if self.evaluate_end_game_phase:
            return self.__h(state.game_board)
        elif self.normalize_result:
            raw_eval = self.__h(state.game_board)
            return self.__normalize(raw_eval)
        else:
            return self.__h(state.game_board)

    def h_piccolo(self, board):
        """
        A similar function to h(), but operates directly on a chess board and allows for custom
        normalization bounds.

        :param board: The chess board to evaluate.
        :return: A heuristic value representing the pawn structure.
        """
        # Handles endgame evaluation or normalization based on the board state.
        if self.evaluate_end_game_phase:
            return self.__h(board)
        elif self.normalize_result:
            raw_eval = self.__h(board)
            return self.__normalize(raw_eval, 10, -10)
        else:
            return self.__h(board)

    def __pawn_structure_score(self, pawns, color, board):
        score = 0
        our_pawns_bb = int(pawns)  # Convert to integer bitboard if it's not already
        all_pawns_bb = int(board.pieces(chess.PAWN, chess.WHITE)) | int(board.pieces(chess.PAWN, chess.BLACK))

        # Precompute pawn presence for files using bitwise operations
        pawns_on_file = [bool(our_pawns_bb & self.file_bb[f]) for f in range(8)]

        for square in chess.SquareSet(our_pawns_bb):
            file = chess.square_file(square)
            rank = chess.square_rank(square)

            # Isolated pawns
            if not (pawns_on_file[file - 1] if file > 0 else False) and \
                    not (pawns_on_file[file + 1] if file < 7 else False):
                score -= 20

            # Doubled pawns
            if bin(our_pawns_bb & self.file_bb[file]).count('1') > 1:
                score -= 10

            # Backward pawns
            supported = False
            advance_square = square + self.advance_shifts[color]
            support_squares = [square - 1, square + 1] + \
                              [advance_square - 1, advance_square + 1]

            # Check if the pawn is supported by our other pawns
            for support_sq in support_squares:
                if 0 <= support_sq < 64 and (all_pawns_bb & (1 << support_sq)):
                    supported = True
                    break
            if not supported and 0 <= advance_square < 64 and board.piece_at(advance_square) is None:
                score -= 15

        return score

    def __passed_pawn_score(self, our_pawns, their_pawns, color):
        """
        Calculates a score based on the structure of the pawns for a given color. It considers factors such
        as isolated, doubled, and backward pawns.

        :param color: The color of the pawns to be evaluated (chess.WHITE or chess.BLACK).
        :return: An integer score representing the structural strengths and weaknesses of the pawns.
        """
        score = 0
        their_pawns_bb = int(their_pawns)  # Assicurati che sia un bitboard intero
        for our_pawn in our_pawns:
            file = chess.square_file(our_pawn)
            rank = chess.square_rank(our_pawn)
            passed = True
            if color == chess.WHITE:
                for r in range(rank + 1, 8):
                    if self.file_bb[file] & self.rank_bb[r] & their_pawns_bb:  # Utilizzo bitboard intero
                        passed = False
                        break
            else:
                for r in range(rank - 1, -1, -1):
                    if self.file_bb[file] & self.rank_bb[r] & their_pawns_bb:  # Utilizzo bitboard intero
                        passed = False
                        break
            if passed:
                score += 50

        return score

    def __h(self, board):
        """
        Private method for the raw heuristic evaluation of the pawn structure on the board. It considers
        the overall structure, including both white and black pawns, and computes a combined score.

        :param board: The chess board to evaluate.
        :return: The raw heuristic value representing the overall pawn structure.
        """
        # Special handling for endgame phase.
        if self.evaluate_end_game_phase:
            game_over_eval = None
            # Assign extreme values for checkmate situations.
            if board.is_checkmate():
                outcome = board.outcome()
                if outcome is not None:
                    game_over_eval = float("inf") if outcome.winner else float("-inf")
            # Assign zero for draw situations.
            if board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
                game_over_eval = 0

            if game_over_eval is not None:
                return game_over_eval
        score = 0
        # Get the positions of white and black pawns.
        white_pawns = board.pieces(chess.PAWN, chess.WHITE)
        black_pawns = board.pieces(chess.PAWN, chess.BLACK)

        # Get the positions of white and black pawns.
        score += self.__pawn_structure_score(white_pawns, chess.WHITE, board)
        score -= self.__pawn_structure_score(black_pawns, chess.BLACK, board)

        # Evaluate passed pawn score for both sides.
        score += self.__passed_pawn_score(white_pawns, black_pawns, chess.WHITE)
        score -= self.__passed_pawn_score(black_pawns, white_pawns, chess.BLACK)

        return score

    def __normalize(self, value, maxv=100, minv=-100):
        """
        Normalizes the evaluation value within a specified range.

        :param value: The value to be normalized.
        :param maxv: The maximum value for normalization. Defaults to 100.
        :param minv: The minimum value for normalization. Defaults to -100.
        :return: The normalized value.
        """
        # Normalizes the value within the range from minv to maxv.
        if value >= 0:
            # Normalizes positive values.
            normalized = (value / self.h_max_value) * 100
        else:
            # Normalizes negative values.
            normalized = (value / abs(self.h_min_value)) * 100

        # Limits the normalized value between minv and maxv.
        normalized = max(min(normalized, maxv), minv)
        return normalized


## Evaluate Piece Positions

This class evaluates the positions of pieces on a chessboard. It calculates a score based on the positioning of each piece type according to specific positional tables, especially considering different game phases (e.g., opening, endgame).

In [ ]:
class EvaluatePiecePositions:
    """
    This class evaluates the positions of pieces on a chessboard. It calculates a score based on the
    positioning of each piece type according to specific positional tables, especially considering different
    game phases (e.g., opening, endgame).

    Attributes:
        evaluate_end_game_phase (bool): If true, the evaluation changes for the endgame phase.
        normalize_result (bool): If true, normalizes the evaluation score within a specific range.
        h_max_value (float): Maximum value for normalization.
        h_min_value (float): Minimum value for normalization.
    """

    def __init__(self, evaluate_end_game_phase=False, normalize_result=False):
        """
        Initializes the evaluator with options for endgame evaluation and normalization of results.

        :param evaluate_end_game_phase: Indicates whether to apply a different evaluation strategy for endgame.
        :param normalize_result: Indicates whether to normalize the evaluation score.
        """
        self.evaluate_end_game_phase = evaluate_end_game_phase  # Determines if endgame is evaluated differently.
        self.normalize_result = normalize_result  # Determines if score should be normalized.
        self.h_max_value = 505  # Maximum value for heuristic normalization.
        self.h_min_value = -420  # Minimum value for heuristic normalization.

    def h(self, state: StateChessGame):
        """
        Evaluates the piece positions for the given state of the chess game.

        :param state: The current state of the chess game.
        :return: A heuristic value representing the evaluation of piece positions.
        """
        # Applies endgame evaluation or normalization based on the initialization flags.
        if self.evaluate_end_game_phase:
            return self.__h(state.game_board)
        elif self.normalize_result:
            raw_eval = self.__h(state.game_board)
            return self.__normalize(raw_eval)
        else:
            return self.__h(state.game_board)

    def h_piccolo(self, board):
        """
        A variant of the h() method, working directly on a chess board. It allows for custom normalization bounds.

        :param board: The chess board to evaluate.
        :return: A heuristic value representing the piece positions.
        """
        # Handles endgame evaluation or normalization based on the board state.
        if self.evaluate_end_game_phase:
            return self.__h(board)
        elif self.normalize_result:
            raw_eval = self.__h(board)
            return self.__normalize(raw_eval, 10, -10)
        else:
            return self.__h(board)

    def __evaluate_piece_positions(self, board, piece_table, piece_type, color):
        """
        Evaluates the positions of a specific type of piece on the board based on a predefined table.

        :param board: The chess board to evaluate.
        :param piece_table: A table with positional values for each square of the board.
        :param piece_type: Type of the piece to evaluate.
        :param color: Color of the pieces to evaluate.
        :return: A score based on the positioning of the pieces.
        """
        score = 0
        pieces = board.pieces(piece_type, color)
        # Calculate the score for each piece based on its position.
        for square in pieces:
            piece_position_value = piece_table[square]
            # Adjust score based on the piece color.
            score += piece_position_value if color == chess.WHITE else -piece_position_value
        return score

    def __is_endgame(self, board):
        """
        Determines if the current board state is in the endgame phase.

        :param board: The chess board to evaluate.
        :return: True if it's the endgame phase, False otherwise.
        """
        # Regine
        white_queens = len(board.pieces(chess.QUEEN, chess.WHITE))
        black_queens = len(board.pieces(chess.QUEEN, chess.BLACK))

        # se entrambi i lati non hanno Regine -> endgame phase
        if white_queens == 0 and black_queens == 0:
            return True

        # Minorpieces
        white_bishops = len(board.pieces(chess.BISHOP, chess.WHITE))
        black_bishops = len(board.pieces(chess.BISHOP, chess.BLACK))
        white_knights = len(board.pieces(chess.KNIGHT, chess.WHITE))
        black_knights = len(board.pieces(chess.KNIGHT, chess.BLACK))
        white_minors = white_bishops + white_knights
        black_minors = black_bishops + black_knights

        white_rooks = len(board.pieces(chess.ROOK, chess.WHITE))
        black_rooks = len(board.pieces(chess.ROOK, chess.BLACK))

        # se ogni lato che ha una regina, non ha altri pezzi oppure ha
        # 1 Minorpiece al massimo -> endgame phase
        white_endgame_condition_with_queen = (
                white_queens == 1 and (white_rooks == 0 and white_minors <= 1)
        )
        black_endgame_condition_with_queen = (
                black_queens == 1 and (black_rooks == 0 and black_minors <= 1)
        )

        if (
                (white_endgame_condition_with_queen and black_queens == 0)
                or (black_endgame_condition_with_queen and white_queens == 0)
                or (
                white_endgame_condition_with_queen
                and black_endgame_condition_with_queen
        )
        ):
            return True

        return False

    def __h(self, board):
        """
        Calculates the total score based on the position of pieces on the board.

        :param board: The chess board to evaluate.
        :return: A score representing the overall evaluation of piece positions.
        """
        # Special handling for endgame phase.
        if self.evaluate_end_game_phase:
            game_over_eval = None
            # Assign extreme values for checkmate situations.
            if board.is_checkmate():
                outcome = board.outcome()
                if outcome is not None:
                    game_over_eval = float("inf") if outcome.winner else float("-inf")
            # Assign zero for draw situations.
            if board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
                game_over_eval = 0

            if game_over_eval is not None:
                return game_over_eval
        total_score = 0
        if self.__is_endgame(board):
            king_table_to_use = KING_ENDGAME_TABLE
        else:
            king_table_to_use = KING_INITGAME_TABLE
        total_score += self.__evaluate_piece_positions(board, PAWN_TABLE, chess.PAWN, chess.WHITE)
        total_score += self.__evaluate_piece_positions(board, KNIGHT_TABLE, chess.KNIGHT, chess.WHITE)
        total_score += self.__evaluate_piece_positions(board, BISHOP_TABLE, chess.BISHOP, chess.WHITE)
        total_score += self.__evaluate_piece_positions(board, ROOK_TABLE, chess.ROOK, chess.WHITE)
        total_score += self.__evaluate_piece_positions(board, QUEEEN_TABLE, chess.QUEEN, chess.WHITE)
        total_score += self.__evaluate_piece_positions(board, king_table_to_use, chess.KING, chess.WHITE)

        total_score -= self.__evaluate_piece_positions(board, PAWN_TABLE, chess.PAWN, chess.BLACK)
        total_score -= self.__evaluate_piece_positions(board, KNIGHT_TABLE, chess.KNIGHT, chess.BLACK)
        total_score -= self.__evaluate_piece_positions(board, BISHOP_TABLE, chess.BISHOP, chess.BLACK)
        total_score -= self.__evaluate_piece_positions(board, ROOK_TABLE, chess.ROOK, chess.BLACK)
        total_score -= self.__evaluate_piece_positions(board, QUEEEN_TABLE, chess.QUEEN, chess.BLACK)
        total_score -= self.__evaluate_piece_positions(board, king_table_to_use, chess.KING, chess.BLACK)

        return total_score

    def __normalize(self, value, maxv=100, minv=-100):
        """
        Normalizes the evaluation value within a specified range.

        :param value: The value to be normalized.
        :param maxv: The maximum value for normalization. Defaults to 100.
        :param minv: The minimum value for normalization. Defaults to -100.
        :return: The normalized value.
        """
        # Normalizes the value within the range from minv to maxv.
        if value >= 0:
            # Normalizes positive values.
            normalized = (value / self.h_max_value) * 100
        else:
            # Normalizes negative values.
            normalized = (value / abs(self.h_min_value)) * 100

        # Limits the normalized value between minv and maxv.
        normalized = max(min(normalized, maxv), minv)
        return normalized


## HardBoardEvaluationChessGame

This class provides an in-depth evaluation of a chess game board by integrating various evaluation metrics like material balance, central control, king safety, mobility, pawn structure, and piece positioning. It offers a holistic approach to assessing the strengths and weaknesses of a chess position, making it suitable for advanced analysis.

Combined heuristics:
   - evaluate_board_without_king: Evaluation component focusing on the board without considering the king's position.
   - evaluate_central_control_score: Evaluation component focusing on the control of central squares.
   - evaluate_king_safety: Evaluation component focusing on the safety of the king.
   - evaluate_mobility: Evaluation component focusing on the mobility of pieces.
   - evaluate_pawn_structure: Evaluation component focusing on the pawn structure.
   - evaluate_piece_positions: Evaluation component focusing on the positions of all pieces except the king.

In [ ]:
class HardBoardEvaluationChessGame:
    """
    This class provides an in-depth evaluation of a chess game board by integrating various evaluation
    metrics like material balance, central control, king safety, mobility, pawn structure, and piece
    positioning. It offers a holistic approach to assessing the strengths and weaknesses of a chess
    position, making it suitable for advanced analysis.

    Attributes:
        balance_evaluation (bool): If True, combines evaluation scores using a weighted approach.
                                   This gives a balanced evaluation considering all aspects of the game.
        print_evaluation (bool): If True, prints the evaluation results for debugging or analysis purposes.
        evaluate_board_without_king (EvaluateBoardWithoutKing): Evaluation component focusing on the board
                                                                without considering the king's position.
        evaluate_central_control_score (EvaluateCentralControlScore): Evaluation component focusing on
                                                                      the control of central squares.
        evaluate_king_safety (EvaluateKingSafety): Evaluation component focusing on the safety of the king.
        evaluate_mobility (EvaluateMobility): Evaluation component focusing on the mobility of pieces.
        evaluate_pawn_structure (EvaluatePawnStructure): Evaluation component focusing on the pawn structure.
        evaluate_piece_positions (EvaluatePiecePositions): Evaluation component focusing on the positions
                                                           of all pieces except the king.
        """

    def __init__(self, balance_evaluation=True, print_evaluation=False):
        """
        Initializes the evaluation components for different aspects of the chess game.

        :param balance_evaluation: If True, uses a weighted approach for combining different evaluation metrics.
        :param print_evaluation: If True, prints the evaluation scores.
        """
        self.balance_evaluation = balance_evaluation  # Flag to use weighted evaluation scores.
        self.print_evaluation = print_evaluation  # Flag to print the evaluation results.
        # Initialize individual evaluation components with normalization enabled.
        self.evaluate_board_without_king = EvaluateBoardWithoutKing(normalize_result=True)
        self.evaluate_central_control_score = EvaluateCentralControlScore(normalize_result=True)
        self.evaluate_king_safety = EvaluateKingSafety(normalize_result=True)
        self.evaluate_mobility = EvaluateMobility(normalize_result=True)
        self.evaluate_pawn_structure = EvaluatePawnStructure(normalize_result=True)
        self.evaluate_piece_positions = EvaluatePiecePositions(normalize_result=True)

    def h(self, state: StateChessGame):
        """
        Evaluates the chess board state using various evaluation metrics.

        :param state: The current state of the chess game.
        :return: A combined heuristic score representing the board evaluation.
        """
        board = state.game_board
        # Special handling for endgame phase.
        game_over_eval = None
        # Assign extreme values for checkmate situations.
        if board.is_checkmate():
            outcome = board.outcome()
            if outcome is not None:
                if outcome.winner:
                    game_over_eval = float("inf")
                else:
                    game_over_eval = float("-inf")
        # Assign zero for draw situations.
        if board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
            game_over_eval = 0

        if game_over_eval is not None:
            return game_over_eval
        else:
            # Evaluate different aspects of the board.
            board_without_king = self.evaluate_board_without_king.h(state)
            central_control_score = self.evaluate_central_control_score.h(state)
            king_safety = self.evaluate_king_safety.h(state)
            mobility = self.evaluate_mobility.h(state)
            pawn_structure = self.evaluate_pawn_structure.h(state)
            piece_positions = self.evaluate_piece_positions.h(state)
            # Optionally print the evaluation scores.
            if self.print_evaluation:
                print("Valutazione: ", board_without_king, central_control_score, king_safety, mobility, pawn_structure,
                      piece_positions)
            # Combine the scores either using weighted or simple sum approach.
            if self.balance_evaluation:
                return (
                        board_without_king * 0.35 +  # Bilancio del materiale
                        central_control_score * 0.20 +  # Controllo del centro
                        king_safety * 0.15 +  # Sicurezza del re
                        mobility * 0.10 +  # Mobilità
                        pawn_structure * 0.10 +  # Struttura dei pedoni
                        piece_positions * 0.10  # Posizione dei pezzi
                )
            return (
                    board_without_king +
                    central_control_score +
                    king_safety +
                    mobility +
                    pawn_structure +
                    piece_positions
            )


## SoftBoardEvaluationChessGame
This class provides a more nuanced evaluation of a chess game board by combining various aspects such as material balance, central control, king safety, and piece positions. It offers a more balanced perspective on the chessboard, making it suitable for a comprehensive game analysis.

Combined heuristics:
   - evaluate_board_without_king: Evaluation component focusing on the board without considering the king's position.
   - evaluate_central_control_score: Evaluation component focusing on the control of central squares.
   - evaluate_king_safety: Evaluation component focusing on the safety of the king.
   - evaluate_piece_positions: Evaluation component focusing on the positions of all pieces except the king.

In [7]:
class SoftBoardEvaluationChessGame:
    """
    This class provides a more nuanced evaluation of a chess game board by combining various aspects such
    as material balance, central control, king safety, and piece positions. It offers a more balanced
    perspective on the chessboard, making it suitable for a comprehensive game analysis.

    Attributes:
        balance_evaluation (bool): If True, uses a weighted approach for combining different evaluation metrics.
        print_evaluation (bool): If True, prints the evaluation scores for debugging or analysis purposes.
        evaluate_board_without_king (EvaluateBoardWithoutKing): Component for evaluating the board without considering the king's position.
        evaluate_central_control_score (EvaluateCentralControlScore): Component for evaluating central control.
        evaluate_king_safety (EvaluateKingSafety): Component for evaluating king safety.
        evaluate_piece_positions (EvaluatePiecePositions): Component for evaluating piece positions.
    """

    def __init__(self, balance_evaluation=True, print_evaluation=False):
        """
        Initializes the evaluation components for different aspects of the chess game.

        :param balance_evaluation: Indicates whether to use a weighted combination of evaluation metrics.
        :param print_evaluation: Indicates whether to print the evaluation scores for each component.
        """
        self.balance_evaluation = balance_evaluation  # Flag for using weighted evaluation scores.
        self.print_evaluation = print_evaluation  # Flag for printing the evaluation results.
        # Initialize individual evaluation components with normalization.
        self.evaluate_board_without_king = EvaluateBoardWithoutKing(normalize_result=True)
        self.evaluate_central_control_score = EvaluateCentralControlScore(normalize_result=True)
        self.evaluate_king_safety = EvaluateKingSafety(normalize_result=True)
        self.evaluate_piece_positions = EvaluatePiecePositions(normalize_result=True)

    def h(self, state: StateChessGame):
        """
        Evaluates the chess board state using a combination of various evaluation metrics.

        :param state: The current state of the chess game.
        :return: A comprehensive heuristic score representing the board evaluation.
        """
        board = state.game_board
        # Handle special game-over conditions like checkmate and stalemate.
        game_over_eval = None
        if board.is_checkmate():
            outcome = board.outcome()
            if outcome is not None:
                if outcome.winner:
                    game_over_eval = float("inf")
                else:
                    game_over_eval = float("-inf")
        if board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
            game_over_eval = 0

        if game_over_eval is not None:
            return game_over_eval
        else:
            # Calculate individual evaluation scores.
            board_without_king = self.evaluate_board_without_king.h(state)
            central_control_score = self.evaluate_central_control_score.h(state)
            king_safety = self.evaluate_king_safety.h(state)
            piece_positions = self.evaluate_piece_positions.h(state)

            # Optionally print the evaluation scores.
            if self.print_evaluation:
                print("Valutazione: ", board_without_king, central_control_score, king_safety, piece_positions)

            # Combine the scores using weighted or simple sum approach.
            if self.balance_evaluation:
                return (
                        board_without_king * 0.35 +  # Bilancio del materiale
                        central_control_score * 0.20 +  # Controllo del centro
                        king_safety * 0.25 +  # Sicurezza del re
                        piece_positions * 0.20  # Posizione dei pezzi
                )
            return (
                    board_without_king +
                    central_control_score +
                    king_safety +
                    piece_positions
            )



## Observation Board

This class observes various aspects of a chess game board, offering a comprehensive analysis of material, space, piece activity, direct threats, and specific evaluations from various components like king safety, mobility, pawn structure, and piece positions.

In [ ]:
from .EvaluateBoardWithoutKing import EvaluateBoardWithoutKing
from .EvaluateCentralControlScore import EvaluateCentralControlScore
from .EvaluateKingSafety import EvaluateKingSafety
from .EvaluateMobility import EvaluateMobility
from .EvaluatePawnStructure import EvaluatePawnStructure
from .EvaluatePiecePositions import EvaluatePiecePositions
from .constants import *


class ObservationBoard:
    """
    This class observes various aspects of a chess game board, offering a comprehensive analysis of material,
    space, piece activity, direct threats.

    Attributes:
        normalize_result (bool): If true, normalizes the evaluation score within a specific range.
    """

    def __init__(self, normalize_result=False):
        """
        Initializes the observation components for different aspects of the chess game.

        :param normalize_result: Indicates whether to normalize the evaluation scores.
        """
        self.normalize_result = normalize_result

    # 87 microsecondi
    def calcola_materiale_totale_spazio_attivita_pezzi_minacce_dirette(self, board):
        """
        Calculates and evaluates several aspects of the chessboard for both players: the total material value,
        space control, activity of pieces (excluding pawns), and the number of direct threats (attacked pieces).

        :param board: The chess board to be evaluated.
        :return: A tuple containing the total material value, space control, piece activity, and direct threats
                 for both players (white and black).
        """
        materiale_bianco = 0  # Total material value for white player.
        materiale_nero = 0  # Total material value for black player.
        spazio_bianco = 0  # Number of squares controlled by white player.
        spazio_nero = 0  # Number of squares controlled by black player.
        attivita_pezzi_bianco = 0  # Activity of pieces (excluding pawns) for white player.
        attivita_pezzi_nero = 0  # Activity of pieces (excluding pawns) for black player.
        minacce_dirette_bianco = 0  # Number of white pieces under direct threat.
        minacce_dirette_nero = 0  # Number of black pieces under direct threat.

        attaccanti_bianco = set()  # Set of squares attacked by white player.
        attaccanti_nero = set()  # Set of squares attacked by black player.

        # Calculate space control and attackers for each square.
        for square in chess.SQUARES:
            if board.is_attacked_by(chess.WHITE, square):
                spazio_bianco += 1
                attaccanti_bianco.add(square)
            if board.is_attacked_by(chess.BLACK, square):
                spazio_nero += 1
                attaccanti_nero.add(square)

        # Evaluate material, piece activity, and direct threats.
        for square in chess.SQUARES:
            pezzo = board.piece_at(square)
            if pezzo:
                valore = PIECE_VALUE[pezzo.piece_type]  # Value of the piece based on its type.
                if pezzo.color == chess.WHITE:
                    materiale_bianco += valore
                    if square in attaccanti_nero:
                        minacce_dirette_bianco += 1
                    if pezzo.piece_type != chess.PAWN:
                        attivita_pezzi_bianco += len(board.attacks(square))
                else:
                    materiale_nero += valore
                    if square in attaccanti_bianco:
                        minacce_dirette_nero += 1
                    if pezzo.piece_type != chess.PAWN:
                        attivita_pezzi_nero += len(board.attacks(square))

        # Return a tuple with the calculated values.
        return (materiale_bianco, materiale_nero, spazio_bianco, spazio_nero, attivita_pezzi_bianco,
                attivita_pezzi_nero, minacce_dirette_bianco, minacce_dirette_nero)

    # 2.55 microsecondi
    def calcola_sicurezza_re(self, board):
        """Calcola la sicurezza del re per ciascun giocatore su una scacchiera di chess."""
        sicurezza_re_bianco = 0
        sicurezza_re_nero = 0

        posizione_re_bianco = board.king(chess.WHITE)
        posizione_re_nero = board.king(chess.BLACK)

        pedone_bianco = chess.Piece(chess.PAWN, chess.WHITE)
        pedone_nero = chess.Piece(chess.PAWN, chess.BLACK)

        # Direzioni per i pedoni bianchi e neri
        direzioni_bianche = [8, 7, 9]  # Nord, Nord-Ovest, Nord-Est
        direzioni_nere = [-8, -7, -9]  # Sud, Sud-Est, Sud-Ovest

        # Calcolare la sicurezza basandosi sui pedoni circostanti e la posizione
        for direzione in direzioni_bianche:
            casa_pedone_bianco = posizione_re_bianco + direzione
            if casa_pedone_bianco in chess.SQUARES and board.piece_at(casa_pedone_bianco) == pedone_bianco:
                sicurezza_re_bianco += 1

        for direzione in direzioni_nere:
            casa_pedone_nero = posizione_re_nero + direzione
            if casa_pedone_nero in chess.SQUARES and board.piece_at(casa_pedone_nero) == pedone_nero:
                sicurezza_re_nero += 1

        return sicurezza_re_bianco, sicurezza_re_nero

    # 6.41 microsecondi
    def calcola_controllo_centro(self, board):
        """
        Calculates the control of the board's center by each player. It assesses which player controls
        the central squares (D4, E4, D5, E5) and by how much, providing an indication of the central dominance
        in the chess game.

        :param board: The chess board to be evaluated.
        :return: A tuple containing the control of the center score for both the white and black players.
        """
        case_centrali = [chess.D4, chess.E4, chess.D5, chess.E5]  # Central squares of the chessboard.
        controllo_centro_bianco = 0  # Control of the center by white player.
        controllo_centro_nero = 0  # Control of the center by black player.

        # Iterate through each central square to assess control by white and black.
        for casa in case_centrali:
            attaccanti_bianchi = board.attackers(chess.WHITE, casa)
            attaccanti_neri = board.attackers(chess.BLACK, casa)

            if attaccanti_bianchi:
                controllo_centro_bianco += 1  # Increase score if white controls the square.
            if attaccanti_neri:
                controllo_centro_nero += 1  # Increase score if black controls the square.

        # Return the control scores for white and black players.
        return controllo_centro_bianco, controllo_centro_nero

    # 3.4 microsecondi
    def calcola_mossa_pedoni(self, board):
        """
        Calculates the number of pawns that have moved from their initial position for each player. This metric
        gives an insight into the pawn advancement and structure in the game, which is a key aspect of chess strategy.

        :param board: The chess board to be evaluated.
        :return: A tuple containing the count of moved pawns for both white and black players.
        """
        mossa_pedoni_bianco = 0  # Number of white pawns that have moved from their initial positions.
        mossa_pedoni_nero = 0  # Number of black pawns that have moved from their initial positions.

        # Iterate through each column to check if pawns have moved from their starting rows.
        for colonna in range(8):
            # Check for white pawns on the second rank.
            casa_iniziale_bianco = chess.square(colonna, 1)
            pezzo_bianco = board.piece_at(casa_iniziale_bianco)
            if not (pezzo_bianco and pezzo_bianco.piece_type == chess.PAWN and pezzo_bianco.color == chess.WHITE):
                mossa_pedoni_bianco += 1

            # Check for black pawns on the seventh rank.
            casa_iniziale_nero = chess.square(colonna, 6)
            pezzo_nero = board.piece_at(casa_iniziale_nero)
            if not (pezzo_nero and pezzo_nero.piece_type == chess.PAWN and pezzo_nero.color == chess.BLACK):
                mossa_pedoni_nero += 1

        # Return the count of moved pawns for both white and black players.
        return mossa_pedoni_bianco, mossa_pedoni_nero

    # 10.2 microsecondi
    def calcola_struttura_pedoni(self, board):
        """
        Calculates a score based on the pawn structure for each player. This evaluation considers aspects like
        isolated and doubled pawns, which are crucial for understanding the pawn dynamics and structural weaknesses
        or strengths in a chess game.

        :param board: The chess board to be evaluated.
        :return: A tuple containing the pawn structure score for both the white and black players.
        """
        punteggio_pedoni_bianco = 0  # Score for the pawn structure of white player.
        punteggio_pedoni_nero = 0  # Score for the pawn structure of black player.
        colonna_pedoni_bianchi = [0] * 8  # Count of white pawns in each column.
        colonna_pedoni_neri = [0] * 8  # Count of black pawns in each column.

        # Count the number of pawns in each column for each player.
        for square in chess.SQUARES:
            pezzo = board.piece_at(square)
            if pezzo and pezzo.piece_type == chess.PAWN:
                colonna = chess.square_file(square)
                if pezzo.color == chess.WHITE:
                    colonna_pedoni_bianchi[colonna] += 1
                else:
                    colonna_pedoni_neri[colonna] += 1

        # Calculate scores based on isolated and doubled pawns.
        for i in range(8):
            # Subtract points for doubled pawns.
            punteggio_pedoni_bianco -= colonna_pedoni_bianchi[i] if colonna_pedoni_bianchi[i] > 1 else 0
            punteggio_pedoni_nero -= colonna_pedoni_neri[i] if colonna_pedoni_neri[i] > 1 else 0

            # Subtract points for isolated pawns.
            if colonna_pedoni_bianchi[i] > 0:
                punteggio_pedoni_bianco -= (i == 0 or colonna_pedoni_bianchi[i - 1] == 0) and (
                        i == 7 or colonna_pedoni_bianchi[i + 1] == 0)
            if colonna_pedoni_neri[i] > 0:
                punteggio_pedoni_nero -= (i == 0 or colonna_pedoni_neri[i - 1] == 0) and (
                        i == 7 or colonna_pedoni_neri[i + 1] == 0)

        # Return the pawn structure score for both white and black players.
        return abs(punteggio_pedoni_bianco), abs(punteggio_pedoni_nero)

    # 1.74 microsecondo
    def calcola_mossa_pezzi_maggiori(self, board):
        """
        Calculates the number of moves made by major pieces (rooks, bishops, queens) for each player.
        This metric assesses how many major pieces have moved from their initial positions,
        providing an insight into the player's development and strategy in the game.

        :param board: The chess board to be evaluated.
        :return: A tuple containing the count of major pieces that have moved for both white and black players.
        """
        mossa_pezzi_maggiori_bianco = 0  # Count of white major pieces that have moved.
        mossa_pezzi_maggiori_nero = 0  # Count of black major pieces that have moved.

        # Initial positions of major pieces for white player.
        posizioni_iniziali_bianco = [chess.A1, chess.B1, chess.C1, chess.D1, chess.E1, chess.F1, chess.G1, chess.H1]
        # Initial positions of major pieces for black player.
        posizioni_iniziali_nero = [chess.A8, chess.B8, chess.C8, chess.D8, chess.E8, chess.F8, chess.G8, chess.H8]

        # Check if major pieces have moved from their initial positions.
        for posizione in posizioni_iniziali_bianco:
            pezzo = board.piece_at(posizione)
            if pezzo is None or (
                    pezzo.piece_type != chess.ROOK and pezzo.piece_type != chess.BISHOP and pezzo.piece_type != chess.QUEEN):
                mossa_pezzi_maggiori_bianco += 1

        for posizione in posizioni_iniziali_nero:
            pezzo = board.piece_at(posizione)
            if pezzo is None or (
                    pezzo.piece_type != chess.ROOK and pezzo.piece_type != chess.BISHOP and pezzo.piece_type != chess.QUEEN):
                mossa_pezzi_maggiori_nero += 1

        # Return the count of moved major pieces for both white and black players.
        return mossa_pezzi_maggiori_bianco, mossa_pezzi_maggiori_nero

    # 949 nanosecondo
    def calcola_sviluppo_pezzi(self, board):
        """
        Calculates the development of pieces (knights and bishops) for each player. This metric assesses how
        many knights and bishops have moved from their initial positions, providing insight into the early
        game development and piece activity, which are crucial aspects of chess strategy.

        :param board: The chess board to be evaluated.
        :return: A tuple containing the count of developed knights and bishops for both white and black players.
        """
        sviluppo_pezzi_bianco = 0  # Count of white knights and bishops that have moved.
        sviluppo_pezzi_nero = 0  # Count of black knights and bishops that have moved.

        # Initial positions of knights and bishops for white player.
        posizioni_iniziali_bianco = [chess.B1, chess.G1, chess.C1, chess.F1]
        # Initial positions of knights and bishops for black player.
        posizioni_iniziali_nero = [chess.B8, chess.G8, chess.C8, chess.F8]

        # Check if knights and bishops have moved from their initial positions.
        for posizione in posizioni_iniziali_bianco:
            pezzo = board.piece_at(posizione)
            if pezzo is None or (pezzo.piece_type != chess.KNIGHT and pezzo.piece_type != chess.BISHOP):
                sviluppo_pezzi_bianco += 1

        for posizione in posizioni_iniziali_nero:
            pezzo = board.piece_at(posizione)
            if pezzo is None or (pezzo.piece_type != chess.KNIGHT and pezzo.piece_type != chess.BISHOP):
                sviluppo_pezzi_nero += 1

        # Return the count of developed knights and bishops for both white and black players.
        return sviluppo_pezzi_bianco, sviluppo_pezzi_nero

    def __normalize(self, valore, max_val, min_val):
        if max_val - min_val == 0:
            return 0  # Evita la divisione per zero se min e max sono uguali
        return (valore - min_val) / (max_val - min_val)

    def h_piccoli(self, board):
        """
        Combines various metrics into a comprehensive array of evaluations for the board. This method
        serves as an aggregator that compiles a wide range of metrics from material value to pawn structure,
        piece activity, and other specialized evaluations, offering a multifaceted view of the board state.

        :param board: The chess board to be evaluated.
        :return: An array containing a diverse set of evaluation results.
        """
        risultati = []  # Array to store the results of various evaluations.

        # Apply normalization if enabled.
        if self.normalize_result:
            # Calculate and normalize various metrics.
            res = self.calcola_materiale_totale_spazio_attivita_pezzi_minacce_dirette(board)
            risultati.append(self.__normalize(res[0], 48, 0))
            risultati.append(self.__normalize(res[1], 48, 0))
            risultati.append(self.__normalize(res[2], 57, 0))
            risultati.append(self.__normalize(res[3], 57, 0))
            risultati.append(self.__normalize(res[4], 84, 0))
            risultati.append(self.__normalize(res[5], 84, 0))
            risultati.append(self.__normalize(res[6], 12, 0))
            risultati.append(self.__normalize(res[7], 12, 0))

            res = self.calcola_sicurezza_re(board)
            risultati.append(self.__normalize(res[0], 4, 0))
            risultati.append(self.__normalize(res[1], 4, 0))

            res = self.calcola_controllo_centro(board)
            risultati.append(self.__normalize(res[0], 5, 0))
            risultati.append(self.__normalize(res[1], 5, 0))

            res = self.calcola_mossa_pedoni(board)
            risultati.append(self.__normalize(res[0], 9, 0))
            risultati.append(self.__normalize(res[1], 9, 0))

            res = self.calcola_struttura_pedoni(board)
            risultati.append(self.__normalize(res[0], 11, 0))
            risultati.append(self.__normalize(res[1], 11, 0))

            res = self.calcola_mossa_pezzi_maggiori(board)
            risultati.append(self.__normalize(res[0], 9, 0))
            risultati.append(self.__normalize(res[1], 9, 0))

            res = self.calcola_sviluppo_pezzi(board)
            risultati.append(self.__normalize(res[0], 5, 0))
            risultati.append(self.__normalize(res[1], 5, 0))
        else:
            # Directly calculate and store the results without normalization.
            risultati.extend(self.calcola_materiale_totale_spazio_attivita_pezzi_minacce_dirette(board))
            risultati.extend(self.calcola_sicurezza_re(board))
            risultati.extend(self.calcola_controllo_centro(board))
            risultati.extend(self.calcola_mossa_pedoni(board))
            risultati.extend(self.calcola_struttura_pedoni(board))
            risultati.extend(self.calcola_mossa_pezzi_maggiori(board))
            risultati.extend(self.calcola_sviluppo_pezzi(board))

        # Return the resulting array.
        return risultati


# Search Algorithm
## MinMaxAlphaBetaPruning
Implements the Minimax algorithm with Alpha-Beta pruning for a chess game.

In [10]:
class MinMaxAlphaBetaPruning:
    """
    Implements the Minimax algorithm with Alpha-Beta pruning for a chess game.

    Attributes:
        game (StateChessGame): The current state of the chess game.
        heuristic (function): Heuristic function used to evaluate game states.
        max_depth (int): Maximum depth for the Minimax search.
        prune_count (int): Count of the number of pruned branches.
        eval_count (int): Count of the number of evaluations performed.
    """

    def __init__(self, game, heuristic, max_depth=1):
        """
        Initializes the MinMaxAlphaBetaPruning class with a game, heuristic function, and maximum search depth.
        :param game: The current state of the chess game.
        :param heuristic: Heuristic function used for evaluating game states.
        :param max_depth: Maximum depth for the Minimax search. Defaults to 1.
        """
        self.game = game  # The current state of the chess game.
        self.heuristic = heuristic  # Heuristic function used to evaluate game states.
        self.max_depth = max_depth  # Maximum depth for the Minimax search.
        self.prune_count = 0  # Count of the number of pruned branches.
        self.eval_count = 0  # Count of the number of evaluations performed.

    def pick(self, states, parent_turn):
        """
        Selects the best state from a list of states based on the player's turn.

        :param states: A list of game states to evaluate.
        :param parent_turn: A flag indicating if it's the parent player's turn.
        :return: The state with the maximum (or minimum) heuristic value based on the player's turn.
        """
        # If it's the parent's turn, choose the state with the maximum heuristic value.
        if parent_turn:
            return max(states, key=lambda state: state.h)
        # If it's the opponent's turn, choose the state with the minimum heuristic value.
        else:
            return min(states, key=lambda state: state.h)

    def evaluate(self, states, parent_turn):
        """
        Evaluates a list of states and updates their heuristic values.

        :param states: A list of game states to evaluate.
        :param parent_turn: A flag indicating if it's the parent player's turn.
        """
        for state in states:
            # If a draw can be claimed in the current state, set heuristic value to 0.0.
            if state.game_board.can_claim_draw():
                state.h = 0.0
            else:
                # Otherwise, evaluate the state using the Minimax algorithm with Alpha-Beta pruning.
                state.h = self.__minmax_alpha_beta(state, self.max_depth - 1, float("-inf"), float("inf"),
                                                   not parent_turn)

    def __minmax_alpha_beta(self, state, depth, alpha, beta, turn):
        """
        Private method implementing the Minimax algorithm with Alpha-Beta pruning.

        :param state: The current game state.
        :param depth: The current depth in the game tree.
        :param alpha: The alpha value for Alpha-Beta pruning.
        :param beta: The beta value for Alpha-Beta pruning.
        :param turn: Flag indicating if it's the maximizing player's turn.
        :return: The heuristic value of the state.
        """
        self.eval_count += 1

        # Base case: if maximum depth is reached or the game is over, return the heuristic value of the state.
        if depth == 0 or state.game_board.is_game_over():
            return self.heuristic.h(state)

        # Generate all possible moves (neighbors) from the current state.
        neighbors = self.game.neighbors(state)

        if turn:  # If it's the maximizing player's turn.
            value = float("-inf")
            for neighbor in neighbors:
                # Recursively call the function to evaluate the neighbor state, updating the value and alpha.
                value = max(value, self.__minmax_alpha_beta(neighbor, depth - 1, alpha, beta, False))
                alpha = max(alpha, value)
                # Alpha-Beta pruning: if alpha is greater or equal to beta, prune this branch.
                if alpha >= beta:
                    self.prune_count += 1
                    break
            return value
        else:  # If it's the minimizing player's turn.
            value = float("inf")
            for neighbor in neighbors:
                # Similarly, for the minimizing player, update the value and beta.
                value = min(value, self.__minmax_alpha_beta(neighbor, depth - 1, alpha, beta, True))
                beta = min(beta, value)
                # Alpha-Beta pruning: if beta is less or equal to alpha, prune this branch.
                if beta <= alpha:
                    self.prune_count += 1
                    break
            return value

    def search(self, state: StateChessGame):
        """
        Public method to start the Minimax search with Alpha-Beta pruning from a given state.

        :param state: The current state of the chess game.
        :return: The best next state for the current player.
        """
        # Generate all possible moves (neighbors) from the current state.
        neighbors = self.game.neighbors(state)
        # Evaluate the neighbors to update their heuristic values.
        self.evaluate(neighbors, state.game_board.turn)
        # Choose the best move based on the current player's turn.
        return self.pick(neighbors, state.game_board.turn)


## MinMaxAlphaBetaPruningH0Cut
Extends the Minimax algorithm with Alpha-Beta pruning for a chess game, incorporating an h0 cutoff heuristic.

In [ ]:
class MinMaxAlphaBetaPruningH0Cut:
    """
    Extends the Minimax algorithm with Alpha-Beta pruning for a chess game, incorporating an h0 cutoff heuristic.

    Attributes:
        game (StateChessGame): The current state of the chess game.
        heuristic (function): Main heuristic function for evaluating game states.
        h0_cut (function): Secondary heuristic function used for h0 cutoff.
        k (int): Number of states to consider after applying the h0 cutoff.
        max_depth (int): Maximum depth for the Minimax search.
        prune_count (int): Count of pruned branches in the main search.
        eval_count (int): Count of evaluations in the main search.
        eval_h0_cut_count (int): Count of evaluations for the h0 cutoff.
        prune_h0_cut_count (int): Count of pruned branches due to the h0 cutoff.
        memoization (dict): Dictionary for memoization to store previously calculated states.
    """

    def __init__(self, game, heuristic, h0_cut, k=5, max_depth=1):
        """
        Initializes the MinMaxAlphaBetaPruningH0Cut class with game settings, heuristics, and search parameters.

        :param game: The current state of the chess game.
        :param heuristic: Main heuristic function used for evaluating game states.
        :param h0_cut: Secondary heuristic function used for h0 cutoff.
        :param k: Number of states to consider after applying the h0 cutoff. Defaults to 5.
        :param max_depth: Maximum depth for the Minimax search. Defaults to 1.
        """
        self.game = game  # The current state of the chess game.
        self.heuristic = heuristic  # Main heuristic function used to evaluate game states.
        self.h0_cut = h0_cut  # Secondary heuristic used for the h0 cutoff.
        self.k = k  # Number of states to consider after applying the h0 cutoff.
        self.max_depth = max_depth  # Maximum depth for the Minimax search.
        self.prune_count = 0  # Count of pruned branches in the main search.
        self.eval_count = 0  # Count of evaluations in the main search.
        self.eval_h0_cut_count = 0  # Count of evaluations for the h0 cutoff.
        self.prune_h0_cut_count = 0  # Count of pruned branches due to the h0 cutoff.
        self.memoization = {}  # Dictionary for storing previously calculated states.

    def pick(self, states, parent_turn):
        """
        Selects the best state based on the player's turn.

        :param states: A list of game states to evaluate.
        :param parent_turn: A flag indicating if it's the parent player's turn.
        :return: The state with the maximum (or minimum) heuristic value based on the player's turn.
        """
        # Choose the state with the maximum or minimum heuristic value depending on the player's turn.
        if parent_turn:
            return max(states, key=lambda state: state.h)
        else:
            return min(states, key=lambda state: state.h)

    def evaluate(self, states, parent_turn):
        """
        Evaluates a list of states and updates their heuristic values.

        :param states: A list of game states to evaluate.
        :param parent_turn: A flag indicating if it's the parent player's turn.
        """
        for state in states:
            # If a draw can be claimed, set heuristic value to 0.0.
            if state.game_board.can_claim_draw():
                state.h = 0.0
            else:
                # Otherwise, evaluate using the Minimax algorithm with Alpha-Beta pruning.
                state.h = self.__minmax_alpha_beta(state, self.max_depth - 1, float("-inf"), float("inf"),
                                                   not parent_turn)

    def __minmax_alpha_beta(self, state, depth, alpha, beta, turn):
        """
        Private method implementing the Minimax algorithm with Alpha-Beta pruning and memoization.

        :param state: The current game state.
        :param depth: The current depth in the game tree.
        :param alpha: The alpha value for Alpha-Beta pruning.
        :param beta: The beta value for Alpha-Beta pruning.
        :param turn: Flag indicating if it's the maximizing player's turn.
        :return: The heuristic value of the state.
        """
        self.eval_count += 1

        # Check if the state is already evaluated and stored in memoization.
        if (state, depth, turn) in self.memoization:
            return self.memoization[(state, depth, turn)]

        # Base case: if maximum depth is reached or the game is over, return the heuristic value.
        if depth == 0 or state.game_board.is_game_over():
            return self.heuristic.h(state)

        # Generate possible moves (neighbors), applying the h0 cutoff.
        neighbors = self.game.neighbors(state)
        top_neighbors = self.__h0_cut(neighbors, state.game_board.turn)

        if turn:  # Maximizing player's turn.
            value = float("-inf")
            for neighbor in top_neighbors:
                # Recursively evaluate the state, update value and alpha.
                value = max(value, self.__minmax_alpha_beta(neighbor, depth - 1, alpha, beta, False))
                alpha = max(alpha, value)
                # Alpha-Beta pruning: prune if alpha >= beta.
                if alpha >= beta:
                    self.prune_count += 1
                    break
            self.memoization[(state, depth, turn)] = value
            return value
        else:  # Minimizing player's turn.
            value = float("inf")
            for neighbor in top_neighbors:
                # Similar evaluation for the minimizing player.
                value = min(value, self.__minmax_alpha_beta(neighbor, depth - 1, alpha, beta, True))
                beta = min(beta, value)
                # Prune if beta <= alpha.
                if beta <= alpha:
                    self.prune_count += 1
                    break
            self.memoization[(state, depth, turn)] = value
            return value

    def __h0_cut(self, states, turn):
        """
        Applies the h0 cutoff heuristic to limit the number of states considered.

        :param states: A list of game states.
        :param turn: Flag indicating the current player's turn.
        :return: A list of states after applying the h0 cutoff.
        """
        initial_count = len(states)
        # Evaluate states using the h0 heuristic and count evaluations.
        for state in states:
            state.h0 = self.h0_cut.h(state)
            self.eval_h0_cut_count += 1

        # Sort and select the top k states based on the h0 heuristic value.
        sorted_states = sorted(states, key=lambda state: state.h0, reverse=turn)[:self.k]
        # Count how many states were pruned by this process.
        self.prune_h0_cut_count += initial_count - len(sorted_states)

        return sorted_states

    def search(self, state: StateChessGame):
        """
        Public method to start the search with Alpha-Beta pruning and h0 cutoff.

        :param state: The current state of the chess game.
        :return: The best next state for the current player.
        """
        # Generate possible moves, applying the h0 cutoff.
        neighbors = self.game.neighbors(state)
        top_neighbors = self.__h0_cut(neighbors, state.game_board.turn)
        # Evaluate the top neighbors and choose the best move based on the player's turn.
        self.evaluate(top_neighbors, state.game_board.turn)
        return self.pick(top_neighbors, state.game_board.turn)


## MinMaxAlphaBetaPruningHlCut

Extends the Minimax algorithm with Alpha-Beta pruning for a chess game, incorporating h0 and hl cutoff heuristics.

In [ ]:
class MinMaxAlphaBetaPruningHlCut:
    """
    Extends the Minimax algorithm with Alpha-Beta pruning for a chess game, incorporating h0 and hl cutoff heuristics.

    Attributes:
        game (StateChessGame): The current state of the chess game.
        heuristic (function): Main heuristic function for evaluating game states.
        h0_cut (function): Heuristic function used for the h0 cutoff.
        k (int): Number of states to consider after applying the h0 and hl cutoffs.
        l (int): Depth for the hl cutoff calculation.
        max_depth (int): Maximum depth for the Minimax search.
        prune_count (int): Count of pruned branches in the main search.
        eval_count (int): Count of evaluations in the main search.
        eval_h0_cut_count (int): Count of evaluations for the h0 cutoff.
        prune_h0_cut_count (int): Count of pruned branches due to the h0 cutoff.
        eval_hl_cut_count (int): Count of evaluations for the hl cutoff.
        prune_hl_cut_count (int): Count of pruned branches due to the hl cutoff.
        memoization (dict): Dictionary for memoization to store previously calculated states.
    """

    def __init__(self, game, heuristic, h0_cut, k=5, l=3, max_depth=1):
        """
        Initializes the MinMaxAlphaBetaPruningHlCut class with game settings, heuristics, and search parameters.

        :param game: The current state of the chess game.
        :param heuristic: Main heuristic function used for evaluating game states.
        :param h0_cut: Heuristic function used for the h0 cutoff.
        :param k: Number of states to consider after applying the h0 and hl cutoffs. Defaults to 5.
        :param l: Depth for the hl cutoff calculation. Defaults to 3.
        :param max_depth: Maximum depth for the Minimax search. Defaults to 1.
        """
        self.game = game  # The current state of the chess game.
        self.heuristic = heuristic  # Main heuristic function for evaluating game states.
        self.h0_cut = h0_cut  # Heuristic function used for the h0 cutoff.
        self.k = k  # Number of states to consider after applying the h0 and hl cutoffs.
        self.l = l  # Depth for the hl cutoff calculation.
        self.max_depth = max_depth  # Maximum depth for the Minimax search.
        self.prune_count = 0  # Count of pruned branches in the main search.
        self.eval_count = 0  # Count of evaluations in the main search.
        self.eval_h0_cut_count = 0  # Count of evaluations for the h0 cutoff.
        self.prune_h0_cut_count = 0  # Count of pruned branches due to the h0 cutoff.
        self.eval_hl_cut_count = 0  # Count of evaluations for the hl cutoff.
        self.prune_hl_cut_count = 0  # Count of pruned branches due to the hl cutoff.
        self.memoization = {}  # Dictionary for storing previously calculated states.

    def pick(self, states, parent_turn):
        """
        Selects the best state based on the player's turn.

        :param states: A list of game states to evaluate.
        :param parent_turn: A flag indicating if it's the parent player's turn.
        :return: The state with the maximum (or minimum) heuristic value based on the player's turn.
        """
        # Choose the state with the maximum or minimum heuristic value depending on the player's turn.
        if parent_turn:
            return max(states, key=lambda state: state.h)
        else:
            return min(states, key=lambda state: state.h)

    def evaluate(self, states, parent_turn):
        """
        Evaluates a list of states and updates their heuristic values.

        :param states: A list of game states to evaluate.
        :param parent_turn: A flag indicating if it's the parent player's turn.
        """
        for state in states:
            # If a draw can be claimed, set heuristic value to 0.0.
            if state.game_board.can_claim_draw():
                state.h = 0.0
            else:
                # Evaluate using the Minimax algorithm with Alpha-Beta pruning.
                state.h = self.__minmax_alpha_beta(state, self.max_depth - 1, float("-inf"), float("inf"),
                                                   not parent_turn)

    # Private method implementing the Minimax algorithm with Alpha-Beta pruning and memoization.
    def __minmax_alpha_beta(self, state, depth, alpha, beta, turn):
        """
        Private method implementing the Minimax algorithm with Alpha-Beta pruning and memoization.

        :param state: The current game state.
        :param depth: The current depth in the game tree.
        :param alpha: The alpha value for Alpha-Beta pruning.
        :param beta: The beta value for Alpha-Beta pruning.
        :param turn: Flag indicating if it's the maximizing player's turn.
        :return: The heuristic value of the state.
        """
        self.eval_count += 1

        # Check if the state is already evaluated and stored in memoization.
        if (state, depth, turn) in self.memoization:
            return self.memoization[(state, depth, turn)]

        # Base case: if maximum depth is reached or the game is over, return the heuristic value.
        if depth == 0 or state.game_board.is_game_over():
            return self.heuristic.h(state)

        # Generate possible moves (neighbors), applying the h0 cutoff.
        neighbors = self.game.neighbors(state)
        top_neighbors = self.__h0_cut(neighbors, state.game_board.turn)

        if turn:  # Maximizing player's turn.
            value = float("-inf")
            for neighbor in top_neighbors:
                # Recursively evaluate the state, update value and alpha.
                value = max(value, self.__minmax_alpha_beta(neighbor, depth - 1, alpha, beta, False))
                alpha = max(alpha, value)
                # Alpha-Beta pruning: prune if alpha >= beta.
                if alpha >= beta:
                    self.prune_count += 1
                    break
            self.memoization[(state, depth, turn)] = value
            return value
        else:  # Minimizing player's turn.
            value = float("inf")
            for neighbor in top_neighbors:
                # Similar evaluation for the minimizing player.
                value = min(value, self.__minmax_alpha_beta(neighbor, depth - 1, alpha, beta, True))
                beta = min(beta, value)
                # Prune if beta <= alpha.
                if beta <= alpha:
                    self.prune_count += 1
                    break
            self.memoization[(state, depth, turn)] = value
            return value

    def __h0_cut(self, states, turn):
        """
        Applies the h0 cutoff heuristic to limit the number of states considered.

        :param states: A list of game states.
        :param turn: Flag indicating the current player's turn.
        :return: A list of states after applying the h0 cutoff.
        """
        initial_count = len(states)
        # Evaluate states using the h0 heuristic and count evaluations.
        for state in states:
            state.h0 = self.h0_cut.h(state)
            self.eval_h0_cut_count += 1

        # Sort and select the top k states based on the h0 heuristic value.
        sorted_states = sorted(states, key=lambda state: state.h0, reverse=turn)[:self.k]
        # Count how many states were pruned by this process.
        self.prune_h0_cut_count += initial_count - len(sorted_states)

        return sorted_states

    def __hl_cut(self, states, turn):
        """
        Applies the hl cutoff heuristic to further limit the number of states considered.

        :param states: A list of game states.
        :param turn: Flag indicating the current player's turn.
        :return: A list of states after applying the hl cutoff.
        """
        initial_count = len(states)
        # Evaluate states using a deeper level of the Minimax algorithm (hl cutoff).
        for state in states:
            state.hl = self.__minmax_alpha_beta_hl(state, self.l - 1, float("-inf"), float("inf"), not turn)
        # Sort and select the top k states based on the hl heuristic value.
        sorted_states = sorted(states, key=lambda state: state.hl, reverse=turn)[:self.k]
        # Count how many states were pruned by this process.
        self.prune_hl_cut_count += initial_count - len(sorted_states)
        return sorted_states

    def __minmax_alpha_beta_hl(self, state, depth, alpha, beta, turn):
        """
        Implements a deeper level of the Minimax algorithm for the hl cutoff.

        :param state: The current game state.
        :param depth: The current depth in the game tree.
        :param alpha: The alpha value for Alpha-Beta pruning.
        :param beta: The beta value for Alpha-Beta pruning.
        :param turn: Flag indicating if it's the maximizing player's turn.
        :return: The heuristic value of the state.
        """
        self.eval_hl_cut_count += 1

        # Base case: if maximum depth is reached or the game is over, return the heuristic value from h0_cut.
        if depth == 0 or state.game_board.is_game_over():
            return self.h0_cut.h(state)

        neighbors = self.game.neighbors(state)

        if turn:  # Maximizing player's turn.
            value = float("-inf")
            for neighbor in neighbors:
                # Recursively evaluate the state for hl cutoff, update value and alpha.
                value = max(value, self.__minmax_alpha_beta_hl(neighbor, depth - 1, alpha, beta, False))
                alpha = max(alpha, value)
                # Alpha-Beta pruning for hl cutoff.
                if alpha >= beta:
                    self.prune_hl_cut_count += 1
                    break
            return value
        else:  # Minimizing player's turn.
            value = float("inf")
            for neighbor in neighbors:
                # Similar evaluation for the minimizing player for hl cutoff.
                value = min(value, self.__minmax_alpha_beta_hl(neighbor, depth - 1, alpha, beta, True))
                beta = min(beta, value)
                # Prune if beta <= alpha in hl cutoff.
                if beta <= alpha:
                    self.prune_hl_cut_count += 1
                    break
            return value

    def search(self, state: StateChessGame):
        """
        Public method to start the search with Alpha-Beta pruning, h0, and hl cutoffs.

        :param state: The current state of the chess game.
        :return: The best next state for the current player.
        """
        # Generate possible moves, applying the hl cutoff.
        neighbors = self.game.neighbors(state)
        top_neighbors = self.__hl_cut(neighbors, state.game_board.turn)
        # Evaluate the top neighbors and choose the best move based on the player's turn.
        self.evaluate(top_neighbors, state.game_board.turn)
        return self.pick(top_neighbors, state.game_board.turn)


## MinMaxAlphaBetaPruningHrCut

Implements the Minimax algorithm with Alpha-Beta pruning for a chess game, incorporating a machine learning-based heuristic evaluation (hr cut).

In [ ]:
import joblib
import pandas as pd

from chessgame import StateChessGame
from chessgame.heuristics.ObservationBoard import ObservationBoard


class MinMaxAlphaBetaPruningHrCut:
    """
    Implements the Minimax algorithm with Alpha-Beta pruning for a chess game, incorporating a machine learning-based heuristic evaluation (hr cut).

    Attributes:
        game (StateChessGame): The current state of the chess game.
        heuristic (function): Main heuristic function used to evaluate game states.
        k (int): Number of states to consider after applying the hr (regressor) cutoff.
        max_depth (int): Maximum depth for the Minimax search.
        prune_count (int): Count of pruned branches in the main search.
        eval_count (int): Count of evaluations in the main search.
        eval_hr_cut_count (int): Count of evaluations for the hr cutoff.
        prune_hr_cut_count (int): Count of pruned branches due to the hr cutoff.
        memoization (dict): Dictionary for storing previously calculated states.
        mlp_regressor (joblib model): Loaded machine learning model for regression.
        observation (ObservationBoard): Observation board for normalizing results.
    """

    def __init__(self, game, heuristic, k=5, max_depth=1):
        """
        Initializes the MinMaxAlphaBetaPruningHrCut class with game settings, heuristics, and search parameters.

        :param game: The current state of the chess game.
        :param heuristic: Main heuristic function used for evaluating game states.
        :param k: Number of states to consider after applying the hr cutoff. Defaults to 5.
        :param max_depth: Maximum depth for the Minimax search. Defaults to 1.
        """
        self.game = game  # The current state of the chess game.
        self.heuristic = heuristic  # Main heuristic function used to evaluate game states.
        self.k = k  # Number of states to consider after applying the h0 cutoff.
        self.max_depth = max_depth  # Maximum depth for the Minimax search.
        self.prune_count = 0  # Count of pruned branches in the main search.
        self.eval_count = 0  # Count of evaluations in the main search.
        self.eval_hr_cut_count = 0  # Count of evaluations for the h0 cutoff.
        self.prune_hr_cut_count = 0  # Count of pruned branches due to the h0 cutoff.
        self.memoization = {}  # Dictionary for storing previously calculated states.
        self.mlp_regressor = joblib.load('./chessgame/mlp_regressor_model.joblib')  # Load the ML regressor model.
        self.observation = ObservationBoard(normalize_result=True)  # Initialize the observation board.

    def pick(self, states, parent_turn):
        """
        Selects the best state from a list of states based on the player's turn.

        :param states: A list of game states to evaluate.
        :param parent_turn: A flag indicating if it's the parent player's turn.
        :return: The state with the maximum (or minimum) heuristic value based on the player's turn.
        """
        # Choose the state with the maximum or minimum heuristic value depending on the player's turn.
        if parent_turn:
            return max(states, key=lambda state: state.h)
        else:
            return min(states, key=lambda state: state.h)

    def evaluate(self, states, parent_turn):
        """
        Evaluates a list of states and updates their heuristic values.

        :param states: A list of game states to evaluate.
        :param parent_turn: A flag indicating if it's the parent player's turn.
        """
        for state in states:
            # If a draw can be claimed, set heuristic value to 0.0.
            if state.game_board.can_claim_draw():
                state.h = 0.0
            else:
                # Otherwise, evaluate using the Minimax algorithm with Alpha-Beta pruning.
                state.h = self.__minmax_alpha_beta(state, self.max_depth - 1, float("-inf"), float("inf"),
                                                   not parent_turn)

    def __minmax_alpha_beta(self, state, depth, alpha, beta, turn):
        """
        Private method implementing the Minimax algorithm with Alpha-Beta pruning.

        :param state: The current game state.
        :param depth: The current depth in the game tree.
        :param alpha: The alpha value for Alpha-Beta pruning.
        :param beta: The beta value for Alpha-Beta pruning.
        :param turn: Flag indicating if it's the maximizing player's turn.
        :return: The heuristic value of the state.
        """
        self.eval_count += 1

        # Check if the state is already evaluated and stored in memoization.
        if (state, depth, turn) in self.memoization:
            return self.memoization[(state, depth, turn)]

        # Base case: if maximum depth is reached or the game is over, return the heuristic value.
        if depth == 0 or state.game_board.is_game_over():
            return self.heuristic.h(state)

        # Generate possible moves (neighbors), applying the hr cutoff.
        neighbors = self.game.neighbors(state)
        top_neighbors = self.__hr_cut(neighbors, state.game_board.turn)

        if turn:  # Maximizing player's turn.
            value = float("-inf")
            for neighbor in top_neighbors:
                # Recursively evaluate the state, update value and alpha.
                value = max(value, self.__minmax_alpha_beta(neighbor, depth - 1, alpha, beta, False))
                alpha = max(alpha, value)
                # Alpha-Beta pruning: prune if alpha >= beta.
                if alpha >= beta:
                    self.prune_count += 1
                    break
            self.memoization[(state, depth, turn)] = value
            return value
        else:  # Minimizing player's turn.
            value = float("inf")
            for neighbor in top_neighbors:
                # Similar evaluation for the minimizing player.
                value = min(value, self.__minmax_alpha_beta(neighbor, depth - 1, alpha, beta, True))
                beta = min(beta, value)
                # Prune if beta <= alpha.
                if beta <= alpha:
                    self.prune_count += 1
                    break
            self.memoization[(state, depth, turn)] = value
            return value

    def __hr_cut(self, states, turn):
        """
        Applies the hr cutoff using the ML regressor to limit the number of states considered.

        :param states: A list of game states.
        :param turn: Flag indicating the current player's turn.
        :return: A list of states after applying the hr cutoff.
        """
        initial_count = len(states)

        for state in states:
            observations = self.observation.h_piccoli(state.game_board)  # Get observations from the board.
            state.hr = self.__regressor_eval(observations)  # Evaluate state using the ML regressor.
            self.eval_hr_cut_count += 1

        # Sort and select the top k states based on the hr value.
        sorted_states = sorted(states, key=lambda state: state.hr, reverse=turn)[:self.k]
        # Count how many states were pruned by this process.
        self.prune_hr_cut_count += initial_count - len(sorted_states)

        return sorted_states

    def __regressor_eval(self, observations):
        """
        Evaluates a state using the ML regressor.

        :param observations: The observations extracted from the chess board.
        :return: The predicted value from the ML regressor.
        """
        colonne = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8', 'h9', 'h10',
                   'h11', 'h12', 'h13', 'h14', 'h15', 'h16', 'h17', 'h18', 'h19',
                   'h20']
        df = pd.DataFrame([observations], columns=colonne)
        return self.mlp_regressor.predict(df)[0]  # Predict and return the first value.

    def search(self, state: StateChessGame):
        """
        Public method to start the search with Alpha-Beta pruning and hr cutoff.

        :param state: The current state of the chess game.
        :return: The best next state for the current player.
        """
        # Generate possible moves, applying the h0 cutoff.
        neighbors = self.game.neighbors(state)
        top_neighbors = self.__hr_cut(neighbors, state.game_board.turn)
        # Evaluate the top neighbors and choose the best move based on the player's turn.
        self.evaluate(top_neighbors, state.game_board.turn)
        return self.pick(top_neighbors, state.game_board.turn)


# Method Main

In [11]:
import time


def main_chess_game():
    game = ChessGame()
    heuristic_a1 = HardBoardEvaluationChessGame()
    search_algorithm_a1 = MinMaxAlphaBetaPruning(game=game, heuristic=heuristic_a1, max_depth=2)
    search_algorithm_a2 = MinMaxAlphaBetaPruning(game=game, heuristic=heuristic_a1, max_depth=2)
    state = StateChessGame()
    agent1 = Agent(search_algorithm_a1, state)
    agent2 = Agent(search_algorithm_a2, state)
    turn_agent = 0
    move_agent_1 = 1
    move_agent_2 = 1
    start_time = time.time()
    print("The game of chess begins!")
    print(state.game_representation)
    while not state.is_endgame():
        if turn_agent % 2:
            state = agent2.do_action(state)
            move_agent_2 += 1
            print(state.game_representation)
            print("Agent 2 (BLACK) played the move:", state.move)
            print()
        else:
            state = agent1.do_action(state)
            move_agent_1 += 1
            print(state.game_representation)
            print("Agent 1 (WHITE) played the move:", state.move)
            print()
        turn_agent = turn_agent + 1

        if state is None:
            print("The agent was unable to resolve the issue")
            return

    end_time = time.time()
    print(f"Result in: {(end_time - start_time) * 1000:.2f}ms")
    print(
        f"OUTCOME: {state.game_representation.game_board.outcome().termination.name}")
    if state.game_representation.get_name_winner_player() is not None:
        print(f"Player Win: {state.game_representation.get_name_winner_player().upper()}")
    print(f"Number of Moves       (agent 1 WHITHE): {move_agent_1}")
    print(f"States evaluated      (agent 1 WHITHE): {agent1.search_algorithm.eval_count}")
    print(f"Pruning carried out   (agent 1 WHITHE): {agent1.search_algorithm.prune_count}")
    print()
    print(f"Number of Moves       (agent 2 BLACK): {move_agent_2}")
    print(f"States evaluated      (agent 2 BLACK): {agent2.search_algorithm.eval_count}")
    print(f"Pruning carried out   (agent 2 BLACK): {agent2.search_algorithm.prune_count}")


if __name__ == '__main__':
    main_chess_game()

The game of chess begins!
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R
Agent 1 (WHITE) played the move: g1f3
r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R
Agent 2 (BLACK) played the move: d7d5
r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . . . . P
. . . . . N . .
P P P P P P P .
R N B Q K B . R
Agent 1 (WHITE) played the move: h2h4
r n . q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . . . . P
. . . . . N . b
P P P P P P P .
R N B Q K B . R
Agent 2 (BLACK) played the move: c8h3
r n . q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . . . . P
. . . . . N . R
P P P P P P P .
R N B Q K B . .
Agent 1 (WHITE) played the move: h1h3
r n . q k b n r
